# Basic example

### 1. Import Seamless in IPython or Jupyter 

In [1]:
from seamless.highlevel import Context
ctx = Context()

### 2. Set up a simple Seamless context

In [2]:
def add(a, b):
    return a + b

ctx.a = 10          # ctx.a => Seamless cell
ctx.b = 20          # ctx.b => Seamless cell
ctx.add = add       # ctx.add => Seamless transformer
ctx.add.a = ctx.a
ctx.add.b = ctx.b
ctx.c = ctx.add.result     # ctx.c => Seamless cell
await ctx.computation()
ctx.c.value

<Silk: 30 >

In [3]:
ctx.a += 5 
await ctx.computation()
ctx.c.value

<Silk: 35 >

### 3. Define schemas and validation rules

In [4]:
ctx.add.example.a = 0.0  # declares that add.a must be a number
ctx.add.example.b = 0.0  

def validate(self):
    assert self.a < self.b

ctx.add.add_validator(validate, name="validate")

await ctx.computation()
print(ctx.add.exception)
# Validation passes => exception is None

None


### 4. Create an API for a Seamless cell

In [5]:
def report(self): 
    value = self.unsilk 
    if value is None: 
        print("Sorry, there is no result") 
    else: 
        print("The result is: {}".format(value))

ctx.c.example.report = report
await ctx.computation()
ctx.c.value.report()

The result is: 35


### 5. Mount cells to the file system

In [6]:
ctx.a.celltype = "plain"
ctx.a.mount("a.txt")
ctx.b.celltype = "plain"
ctx.b.mount("b.txt")
ctx.c.celltype = "plain"
ctx.c.mount("c.txt", mode="w")
ctx.add.code.mount("code.py")
await ctx.translation()

### 6. Share a cell over HTTP

In [7]:
ctx.c.mimetype = "text"
ctx.c.share()
await ctx.translation()

### 7. Control cells from Jupyter

In [8]:
from ipywidgets import IntSlider, IntText

a = IntSlider(min=-10,max=30)
b = IntSlider(min=-10,max=30)
c = ctx.c.output()
ctx.a.traitlet().link(a)
ctx.b.traitlet().link(b)
display(a)
display(b)
display(c)

Opened the seamless share update server at port 5138


IntSlider(value=15, max=30, min=-10)

IntSlider(value=20, max=30, min=-10)

Output()

### 8. Display the shared cell in the browser
**Run the two collapsed cells below**


In [9]:
from seamless.shareserver import shareserver
if (shareserver.rest_port, shareserver.update_port) != (5813, 5138):
    print("""WARNING: a different instance of Seamless is already running. 
The URL below will NOT work """)

In [10]:
%%javascript

let base = window.location.protocol + "//" + window.location.hostname
let v = window.location.pathname
let vv = v.split("/")
for (let i = 1; i < vv.length - 1; i++) {
    if (vv[i] == "lab" && vv[i+1] == "tree") {
        var url = base + vv.slice(0, i).join("/") + "/proxy/5813/ctx/c" 
        break
    }
}

let ele = document.createElement("div")
element.append(ele)
ele.innerHTML = "<b><a href=\"" + url  + "\" target=\"_blank\"> Click here to display cell ctx.c in a new tab </a></b>"


<IPython.core.display.Javascript object>

### 9. Save the entire state of the context

In [11]:
# Graph and checksums, as JSON
ctx.save_graph("basic-example.seamless")
# Checksum-to-buffer cache, as ZIP file
ctx.save_zip("basic-example.zip")

### 10. Click below to continue in a new notebook...


<a href=./load-basic-example.ipynb target="_blank">./load-basic-example.ipynb</a>

### 11. Stop the current kernel

In [ ]:
import asyncio
await asyncio.sleep(10)
import os
os._exit(00)

Opened the seamless REST server at port 5813
